In [1]:
from geopy.distance import distance as geo_dist

In [2]:
import socket

if not 'iccluster' in socket.gethostname():
    # set this to the base spark directory on your system
    spark_home = '/path/to/spark'
    try:
        import findspark
        findspark.init(spark_home)
    except ModuleNotFoundError as e:
        print('Info: {}'.format(e))

import getpass
import pyspark

In [3]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Stops extraction") \
    .config("spark.driver.host", "localhost") \
    .getOrCreate()

In [4]:
sc = spark.sparkContext
spark

('8503000', '47.378177', '8.540192', '408', 'Zürich HB')

In [32]:
zurich_geo = (47.378177, 8.540192)

In [41]:
stops = sc.textFile("/datasets/project/metadata/BFKOORD_GEO")
stops = stops.map(lambda x: x.split()).map(lambda x: (int(x[0]), float(x[2]), float(x[1]), int(x[3]), " ".join(x[5:])))
stops = stops.map(lambda x: (*x, geo_dist(zurich_geo, (x[1], x[2])).km ))
stops = stops.filter(lambda x: x[5] <= 10)

In [43]:
stops_df = stops.toDF().toPandas()

In [45]:
stops_df.columns = ['id', 'latitude', 'longitude', 'elevation', 'station_name', 'dist_to_zurich_HB']

In [49]:
stops_df.to_csv("zurich_hb_stops.csv", index=False, encoding='utf-8')